In [1]:
"""
============================================================================
FULL LEAKAGE MODE - TÜM SIZILAR AÇIK!
============================================================================
Hipotez: Makale şu hataları yapmış olabilir:
1. ❌ Normalization BEFORE split
2. ❌ NO LAG (same-day features)
3. ❌ RANDOM shuffle (future data in train)
4. ❌ Look-ahead bias (indicators already contain target info)

Test: Tüm bunları yapalım ve %90 accuracy elde edelim!
============================================================================
"""

import sys
import subprocess
print("📦 Yükleniyor...")
subprocess.check_call([sys.executable, "-m", "pip", "install", "-q",
                      "yfinance", "ta", "scikit-learn", "pandas", "numpy"])

import yfinance as yf
import pandas as pd
import numpy as np
import ta
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix
import warnings
warnings.filterwarnings('ignore')

print("✅ Hazır!\n")

# ============================================================================
# VERİ
# ============================================================================
print("="*80)
print("VERİ ÇEKME - KOSPI")
print("="*80)

ticker = '^KS11'
data = yf.download(ticker, start="2011-01-01", end="2020-09-27",
                  progress=False, auto_adjust=True)

if isinstance(data.columns, pd.MultiIndex):
    data.columns = data.columns.get_level_values(0)

data = data[['Open', 'High', 'Low', 'Close', 'Volume']].copy()
data = data.dropna()
print(f"✅ {len(data)} gün\n")

# ============================================================================
# TEKNİK GÖSTERGELER
# ============================================================================

def calculate_indicators(df):
    df = df.copy()

    high = df['High'].squeeze()
    low = df['Low'].squeeze()
    close = df['Close'].squeeze()

    # Stochastic
    stoch = ta.momentum.StochasticOscillator(high, low, close, window=14, smooth_window=3)
    df['Stochastic_K'] = stoch.stoch()
    df['Stochastic_D'] = stoch.stoch_signal()

    # ROC
    df['ROC'] = ta.momentum.ROCIndicator(close, window=10).roc()

    # Williams %R
    df['Williams_R'] = ta.momentum.WilliamsRIndicator(high, low, close, lbp=14).williams_r()

    # Momentum
    df['Momentum'] = close.diff(4)

    # Disparity
    ma5 = close.rolling(5).mean()
    ma14 = close.rolling(14).mean()
    df['Disparity_5'] = np.where(ma5 != 0, (close / ma5) * 100, 100)
    df['Disparity_14'] = np.where(ma14 != 0, (close / ma14) * 100, 100)

    # OSCP
    ma10 = close.rolling(10).mean()
    df['OSCP'] = np.where(ma5 != 0, ((ma5 - ma10) / ma5), 0)

    # CCI
    df['CCI'] = ta.trend.CCIIndicator(high, low, close, window=20).cci()

    # RSI
    df['RSI'] = ta.momentum.RSIIndicator(close, window=14).rsi()

    # Pivot Points
    prev_high = high.shift(1)
    prev_low = low.shift(1)
    prev_close = close.shift(1)

    df['Pivot_Point'] = (prev_high + prev_low + prev_close) / 3
    df['S1'] = (df['Pivot_Point'] * 2) - prev_high
    df['S2'] = df['Pivot_Point'] - (prev_high - prev_low)
    df['R1'] = (df['Pivot_Point'] * 2) - prev_low
    df['R2'] = df['Pivot_Point'] + (prev_high - prev_low)

    df = df.replace([np.inf, -np.inf], np.nan)
    return df

data = calculate_indicators(data)
print("✅ Göstergeler hesaplandı\n")

# ============================================================================
# EXTREME LEAKAGE SCENARIOS
# ============================================================================

def scenario_1_worst_leakage(df):
    """
    ❌❌❌ EN KÖTÜ SENARYO - TÜM LEAKAGE'LAR AÇIK

    1. NO LAG (bugünün göstergeleri)
    2. Normalize BEFORE split (tüm veriye fit)
    3. RANDOM shuffle (gelecek train'de)
    4. Same-day target (bugünkü yön)
    """
    df = df.copy()

    features = ['Stochastic_K', 'Stochastic_D', 'ROC', 'Williams_R',
                'Momentum', 'Disparity_5', 'Disparity_14', 'OSCP',
                'CCI', 'RSI', 'Pivot_Point', 'S1', 'S2', 'R1', 'R2']

    # ❌ 1. SAME-DAY TARGET (bugünün kapanış yönü)
    df['Target'] = (df['Close'] > df['Close'].shift(1)).astype(int)

    df = df.dropna(subset=features + ['Target'])

    # ❌ 2. NORMALIZE ALL DATA FIRST (leakage!)
    scaler = MinMaxScaler()
    df[features] = scaler.fit_transform(df[features])

    X = df[features].values
    y = df['Target'].values

    # ❌ 3. RANDOM SHUFFLE SPLIT (gelecek verisi görülüyor!)
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, shuffle=True, stratify=y
    )

    return X_train, X_test, y_train, y_test, "EXTREME LEAKAGE (NO LAG + SAME DAY)"


def scenario_2_normalize_before_split(df):
    """
    ❌❌ Normalize BEFORE split + Random shuffle

    1. LAG VAR (t-1 features) ✅
    2. Normalize BEFORE split ❌
    3. RANDOM shuffle ❌
    4. Next-day target ✅
    """
    df = df.copy()

    features = ['Stochastic_K', 'Stochastic_D', 'ROC', 'Williams_R',
                'Momentum', 'Disparity_5', 'Disparity_14', 'OSCP',
                'CCI', 'RSI', 'Pivot_Point', 'S1', 'S2', 'R1', 'R2']

    # Target (next day)
    df['Target'] = (df['Close'].shift(-1) > df['Close']).astype(int)
    df = df.iloc[:-1]

    df = df.dropna(subset=features + ['Target'])

    # ❌ NORMALIZE FIRST (leakage!)
    scaler = MinMaxScaler()
    df[features] = scaler.fit_transform(df[features])

    # Lag apply
    lagged_features = []
    for feat in features:
        lagged_col = f'{feat}_lag1'
        df[lagged_col] = df[feat].shift(1)
        lagged_features.append(lagged_col)

    df = df.dropna(subset=lagged_features)

    X = df[lagged_features].values
    y = df['Target'].values

    # ❌ RANDOM SHUFFLE
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, shuffle=True, stratify=y
    )

    return X_train, X_test, y_train, y_test, "NORMALIZE BEFORE SPLIT + SHUFFLE"


def scenario_3_random_cv_only(df):
    """
    ❌ Random CV (StratifiedKFold shuffle=True)

    1. LAG VAR ✅
    2. Normalize correctly ✅
    3. Temporal split ✅
    4. BUT: Random CV folds ❌
    """
    df = df.copy()

    features = ['Stochastic_K', 'Stochastic_D', 'ROC', 'Williams_R',
                'Momentum', 'Disparity_5', 'Disparity_14', 'OSCP',
                'CCI', 'RSI', 'Pivot_Point', 'S1', 'S2', 'R1', 'R2']

    # Target
    df['Target'] = (df['Close'].shift(-1) > df['Close']).astype(int)
    df = df.iloc[:-1]

    df = df.dropna(subset=features + ['Target'])

    # Lag
    lagged_features = []
    for feat in features:
        lagged_col = f'{feat}_lag1'
        df[lagged_col] = df[feat].shift(1)
        lagged_features.append(lagged_col)

    df = df.dropna(subset=lagged_features)

    X = df[lagged_features].copy()
    y = df['Target'].copy()

    # Temporal split
    n_train = int(len(X) * 0.8)
    X_train = X.iloc[:n_train]
    X_test = X.iloc[n_train:]
    y_train = y.iloc[:n_train].values
    y_test = y.iloc[n_train:].values

    # Normalize correctly
    scaler = MinMaxScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # ❌ BUT use random CV for hyperparameter tuning
    # (This will be shown in cross-validation score)

    return X_train_scaled, X_test_scaled, y_train, y_test, "CORRECT BUT RANDOM CV"


def scenario_4_correct(df):
    """
    ✅ CORRECT METHOD

    Everything done properly
    """
    df = df.copy()

    features = ['Stochastic_K', 'Stochastic_D', 'ROC', 'Williams_R',
                'Momentum', 'Disparity_5', 'Disparity_14', 'OSCP',
                'CCI', 'RSI', 'Pivot_Point', 'S1', 'S2', 'R1', 'R2']

    # Target
    df['Target'] = (df['Close'].shift(-1) > df['Close']).astype(int)
    df = df.iloc[:-1]

    df = df.dropna(subset=features + ['Target'])

    # Lag
    lagged_features = []
    for feat in features:
        lagged_col = f'{feat}_lag1'
        df[lagged_col] = df[feat].shift(1)
        lagged_features.append(lagged_col)

    df = df.dropna(subset=lagged_features)

    X = df[lagged_features].copy()
    y = df['Target'].copy()

    # Temporal split
    n_train = int(len(X) * 0.8)
    X_train = X.iloc[:n_train]
    X_test = X.iloc[n_train:]
    y_train = y.iloc[:n_train].values
    y_test = y.iloc[n_train:].values

    # Normalize correctly
    scaler = MinMaxScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    return X_train_scaled, X_test_scaled, y_train, y_test, "✅ CORRECT METHOD"


# ============================================================================
# EVALUATION
# ============================================================================

def evaluate_scenario(X_train, X_test, y_train, y_test, name):
    """Train and evaluate"""

    print(f"\n{'='*80}")
    print(f"{name}")
    print(f"{'='*80}")
    print(f"Train: {len(X_train)} | Test: {len(X_test)}")
    print(f"Class dist: UP={y_train.mean()*100:.1f}%")

    # Simple SVM (paper's parameters)
    svm = SVC(kernel='linear', C=1.0, random_state=42)

    print("\nTraining SVM...")
    svm.fit(X_train, y_train)

    # Predictions
    y_pred = svm.predict(X_test)

    # Metrics
    acc = accuracy_score(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred)

    print(f"\n{'RESULTS':^80}")
    print("-" * 80)
    print(f"Test Accuracy: {acc:.4f} ({acc*100:.2f}%)")

    print(f"\nConfusion Matrix:")
    print(f"                Predicted DOWN  Predicted UP")
    print(f"Actual DOWN          {cm[0,0]:<8}      {cm[0,1]:<8}")
    print(f"Actual UP            {cm[1,0]:<8}      {cm[1,1]:<8}")

    # Class-wise
    tn, fp, fn, tp = cm.ravel()
    down_acc = tn / (tn + fp) if (tn + fp) > 0 else 0
    up_acc = tp / (tp + fn) if (tp + fn) > 0 else 0

    print(f"\nClass-wise Accuracy:")
    print(f"DOWN: {down_acc:.4f} ({down_acc*100:.1f}%)")
    print(f"UP:   {up_acc:.4f} ({up_acc*100:.1f}%)")
    print(f"Balance: {abs(down_acc - up_acc):.4f}")

    # Verdict
    if acc >= 0.85:
        print(f"\n🎉 PAPER ACCURACY ACHIEVED! ({acc*100:.1f}%)")
    elif acc >= 0.70:
        print(f"\n🟡 HIGH ACCURACY ({acc*100:.1f}%) - Likely data leakage")
    elif acc >= 0.60:
        print(f"\n🟢 GOOD ACCURACY ({acc*100:.1f}%)")
    else:
        print(f"\n🔵 REALISTIC ACCURACY ({acc*100:.1f}%) - No leakage")

    return acc

# ============================================================================
# RUN ALL SCENARIOS
# ============================================================================

print("\n" + "="*80)
print("TESTING ALL LEAKAGE SCENARIOS")
print("="*80)

results = {}

# Scenario 1: EXTREME LEAKAGE
X_train, X_test, y_train, y_test, name = scenario_1_worst_leakage(data)
results['Scenario 1'] = evaluate_scenario(X_train, X_test, y_train, y_test, name)

# Scenario 2: Normalize before split
X_train, X_test, y_train, y_test, name = scenario_2_normalize_before_split(data)
results['Scenario 2'] = evaluate_scenario(X_train, X_test, y_train, y_test, name)

# Scenario 3: Random CV
X_train, X_test, y_train, y_test, name = scenario_3_random_cv_only(data)
results['Scenario 3'] = evaluate_scenario(X_train, X_test, y_train, y_test, name)

# Scenario 4: CORRECT
X_train, X_test, y_train, y_test, name = scenario_4_correct(data)
results['Scenario 4'] = evaluate_scenario(X_train, X_test, y_train, y_test, name)

# ============================================================================
# SUMMARY
# ============================================================================

print("\n" + "="*80)
print("📊 SUMMARY - ACCURACY COMPARISON")
print("="*80)

print(f"\n{'Scenario':<45} {'Accuracy':<12} {'Status'}")
print("-" * 80)

scenarios = [
    ('Scenario 1: EXTREME LEAKAGE (NO LAG + SAME DAY)', results['Scenario 1']),
    ('Scenario 2: NORMALIZE BEFORE SPLIT + SHUFFLE', results['Scenario 2']),
    ('Scenario 3: CORRECT BUT RANDOM CV', results['Scenario 3']),
    ('Scenario 4: ✅ FULLY CORRECT', results['Scenario 4'])
]

for name, acc in scenarios:
    if acc >= 0.85:
        status = "🎉 PAPER LEVEL"
    elif acc >= 0.70:
        status = "🟡 HIGH (Leakage)"
    elif acc >= 0.60:
        status = "🟢 GOOD"
    else:
        status = "🔵 REALISTIC"

    print(f"{name:<45} {acc*100:>5.2f}%       {status}")

print("\n" + "="*80)
print("💡 CONCLUSIONS")
print("="*80)
print("""
1. If Scenario 1-2 achieves 85-90%, the paper has SEVERE data leakage
2. If Scenario 3 is high but 4 is low, paper used random CV incorrectly
3. If ALL scenarios show ~55-60%, your implementation is CORRECT
   and the paper's methodology is QUESTIONABLE

Your realistic accuracy (~56%) is NORMAL for financial prediction!
Papers reporting 85-90% are almost always using incorrect methodology.
""")

print("\n" + "="*80)
print("✅ ANALYSIS COMPLETE")
print("="*80)

📦 Yükleniyor...
✅ Hazır!

VERİ ÇEKME - KOSPI
✅ 2397 gün

✅ Göstergeler hesaplandı


TESTING ALL LEAKAGE SCENARIOS

EXTREME LEAKAGE (NO LAG + SAME DAY)
Train: 1902 | Test: 476
Class dist: UP=52.3%

Training SVM...

                                    RESULTS                                     
--------------------------------------------------------------------------------
Test Accuracy: 0.7878 (78.78%)

Confusion Matrix:
                Predicted DOWN  Predicted UP
Actual DOWN          163           64      
Actual UP            37            212     

Class-wise Accuracy:
DOWN: 0.7181 (71.8%)
UP:   0.8514 (85.1%)
Balance: 0.1333

🟡 HIGH ACCURACY (78.8%) - Likely data leakage

NORMALIZE BEFORE SPLIT + SHUFFLE
Train: 1900 | Test: 476
Class dist: UP=52.4%

Training SVM...

                                    RESULTS                                     
--------------------------------------------------------------------------------
Test Accuracy: 0.5231 (52.31%)

Confusion Matrix:
     